In [1]:
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.subplots

import numpy as np
import matplotlib.pyplot as plt
import dpx_control_hw as dch

## Settings

In [2]:
CONFIG = '../config.conf'

## Find port and connect

In [3]:
port = dch.find_port()
if port is None:
    port = '/dev/ttyACM0'

dpx = dch.Dosepix(
    port_name=port,
    config_fn=CONFIG,
    thl_calib_fn=None,
    params_fn=None,
    bin_edges_fn=None
)

## ToT-measurement generator

In [10]:
tot_gen = dpx.dpm.measure_tot(
    frame_time=0,
    save_frames=None,
    out_dir=None,
    meas_time=None,
    make_hist=True,
    use_gui=True
)

Exception ignored in: <generator object DPXMeasurement.measure_tot_gen at 0x7f69a043a190>
Traceback (most recent call last):
  File "/tmp/ipykernel_1821875/4284844403.py", line 1, in <cell line: 1>
RuntimeError: generator ignored GeneratorExit


Starting ToT Measurement!


Exception ignored in: <generator object DPXMeasurement.measure_tot_gen at 0x7f69a043a0b0>
Traceback (most recent call last):
  File "/home/solvalou/Documents/dosimetry/dpx_control_hw_venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 1348, in reset
    del ns[k]
RuntimeError: generator ignored GeneratorExit


## Create dash-app

In [6]:
app = JupyterDash(__name__, update_title=None)
app.layout = html.Div(
    [
        dcc.Graph(id='live-update-graph'),
        dcc.Interval(
            id="interval",
            interval=100,
            n_intervals=0
        ),
        html.Div(children=[
            html.Label('ToT-range'),
            dcc.Slider(
                id='tot_slider',
                min=0,
                max=800,
                value=400
            )
        ])
    ]
)

plt_data = {'bins': np.arange(4095), 'hist': np.zeros(4095)}

@app.callback(
    Output('live-update-graph', 'figure'),
    [Input('interval', 'n_intervals')],
    Input('tot_slider', 'value')
)
def update_data(n, tot_slider_value):
    try:
        plt_data['hist'] = np.sum(next(tot_gen), axis=0)
    except ValueError:
        pass
        
    fig = plotly.subplots.make_subplots(vertical_spacing=0.2)
    fig['layout']['margin'] = {
        'l': 30, 'r': 10, 'b': 30, 't': 10
    }
    fig['layout']['legend'] = {'x': 0, 'y': 1, 'xanchor': 'left'}

    fig.append_trace({
        'x': plt_data['bins'][:tot_slider_value],
        'y': plt_data['hist'][:tot_slider_value],
        'name': 'Bias Voltage',
        'mode': 'lines+markers',
        'type': 'scatter'
    }, 1, 1)
    return fig

## Run server

In [7]:
app.run_server(debug=True, mode='external')

Dash app running on http://127.0.0.1:8050/
Starting ToT Measurement!


In [11]:
app._terminate_server_for_port("localhost", 8050)

AttributeError: 'JupyterDash' object has no attribute '_terminate_server_for_port'